# Question Answering

## Overview

Recall the overall workflow for retrieval augmented generation (RAG):

![overview.jpeg](attachment:overview.jpeg)

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB.

In [2]:
!pip install langchain openai chromadb tiktoken
!pip install pypdf
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 894.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:0

In [4]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023.
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [5]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [8]:
# !pip install -U langchain-community

In [20]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from google.colab import drive
# drive.mount('/content/drive')
persist_directory = '/sample_data'
# persist_directory = 'sample_data/The_History_of_Starbucks.pdf'
# embedding = OpenAIEmbeddings()
embedding = OpenAIEmbeddings(
    openai.api_key = os.environ['OPENAI_API_KEY'])
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [21]:
print(vectordb._collection.count())

0


In [29]:
question = "Tell me something about Starbucks?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

0

In [30]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai.api_key = os.environ['OPENAI_API_KEY'])

### RetrievalQA chain

In [31]:
from langchain.chains import RetrievalQA

In [32]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [33]:
result = qa_chain({"query": question})

In [34]:
result["result"]

'Starbucks is a multinational chain of coffeehouses and roastery reserves based in Seattle, Washington. It is known for its wide range of coffee drinks, teas, pastries, and other snacks. Starbucks has locations all around the world and is a popular destination for coffee lovers and those looking for a cozy place to work or socialize.'

### Prompt

In [35]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [36]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [37]:
question = "Who are Starbucks and Pret's founders?"

In [38]:
result = qa_chain({"query": question})

In [39]:
result["result"]

'Starbucks was founded by Jerry Baldwin, Zev Siegl, and Gordon Bowker in 1971. Pret A Manger was founded by Sinclair Beecham and Julian Metcalfe in 1983. Thanks for asking!'

In [41]:
# result["source_documents"][0]

### RetrievalQA chain types

In [42]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [43]:
result = qa_chain_mr({"query": question})

In [44]:
result["result"]

'Starbucks was founded by Jerry Baldwin, Zev Siegl, and Gordon Bowker. Pret A Manger was founded by Julian Metcalfe and Sinclair Beecham.'

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [48]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = os.environ['LANGCHAIN_API_KEY'] # replace dots with your api key

In [49]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'Starbucks was founded by Jerry Baldwin, Zev Siegl, and Gordon Bowker. Pret A Manger was founded by Sinclair Beecham and Julian Metcalfe.'

In [66]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = vectordb.as_retriever()

# New Syntex???
qa_chain_mr = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="map_reduce"
)

# query
question = "Can you tell me something about Starbucks?"
result = qa_chain_mr({"query": question})

# Output the result
print(result["result"])


I'm sorry, I don't know.


In [60]:
# qa_chain_mr = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever(),
#     chain_type="refine"
# )
# result = qa_chain_mr({"query": question})
# result["result"]

### RetrievalQA limitations

QA fails to preserve conversational history.

In [61]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [64]:
question = "What are Starbucks and Pret's reputations? What are their differences?"
result = qa_chain({"query": question})
result["result"]

"Starbucks and Pret A Manger are both popular coffee shop chains known for their coffee and food offerings. Starbucks is known for its wide variety of coffee drinks, cozy atmosphere, and global presence. Pret A Manger is known for its fresh and natural food options, with a focus on sandwiches, salads, and baked goods.\n\nIn terms of reputation, Starbucks is often seen as a convenient and consistent option for coffee and snacks, while Pret A Manger is known for its emphasis on natural ingredients and sustainability practices.\n\nTheir main differences lie in their food offerings and overall atmosphere. Pret A Manger focuses more on fresh and natural food options, while Starbucks offers a wider range of coffee drinks and pastries. Additionally, Pret A Manger tends to have a more minimalist and modern atmosphere compared to Starbucks' cozy and inviting ambiance."

In [65]:
question = "Which cafe is more profitable, Starbucks or Pret?"
result = qa_chain({"query": question})
result["result"]

"I don't have access to the specific financial data of Starbucks and Pret a Manger to determine which cafe is more profitable. You may need to look into their respective financial reports or industry analyses to find this information."

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.